In [1]:
import requests
from flask import Flask, request, jsonify
from flask_cors import CORS
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import Normalizer
import torch
import mglearn
from collections import Counter
from kiwipiepy import Kiwi, TypoTransformer, TypoDefinition
import re
import pandas as pd
import json
import numpy as np
from kiwipiepy.utils import Stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
normalizer = Normalizer()
clustered_dong = pd.read_json("./data/군집상권데이터.json")
menu_data = pd.read_json("./data/메뉴_추상설명_재료_토큰화.json")
cafe_menu_data = pd.read_json("./data/메뉴 데이터.json")

In [ ]:
app = Flask(__name__)
CORS(app)  # 모든 도메인에서의 요청 허용

# CORS(app, resources={r"/api/*": {"origins": "http://localhost:8087"}})
# REST API key
KAKAO_API_KEY = "015412fbc48c3a4e31b1926b6adb667e"


# 스프링 부트로부터 사용자가 클릭한 좌표 정보를 전달받아 동을 찾은 후 분석하고 전송
@app.route('/send-coordinates', methods=['POST'])
def receive_coordinates():
    # 클라이언트로부터 받은 데이터(현 위치 위도, 경도)
    data = request.get_json()
    latitude = data.get('latitude')
    longitude = data.get('longitude')
    
    url = f"https://dapi.kakao.com/v2/local/geo/coord2address.json?x={longitude}&y={latitude}"
    headers = {"Authorization" : f"KakaoAK {KAKAO_API_KEY}"}
    
    response = requests.get(url, headers=headers)
    
    
    dong_number = get_dong_number(latitude, longitude)
    if(dong_number == None):
        return jsonify({"status" : "error", "message" : "현재 지역의 법정동 번호를 찾을 수 없습니다."})
    
    # 유사 상권의 법정동 번호
    similar_dongs = yusa_dong(clustered_dong, dong_number)
    
    # 스트링 부트로 전달
    spring_response = similar_dong_send_to_spring(similar_dongs)
    # 전송 결과 확인
    if(spring_response.get("status")=="success"):
        return jsonify({"status":"success",
                        "yusadongs":similar_dongs,
                        "message": "flask successed"
                       })
    else:
        return jsonify({
            "status" : "error",
            "yusadongs" : "",
            "message" : spring_response.get("error")
        })


# 유사 상권 탐색 2. 스프링으로부터 동을 전달받아 유사한 동 분석
@app.route("/yusa-dong-search", methods=["POST"])
def receive_dong():
    data = request.get_json()
    input_data = data.get("dong").strip()
    dong = "-1"
    region = "-1"
    
    yusa_dong_list = []
    
    if(len(input_data.split(" "))>1):
        dong = input_data.split(" ")[-1]
        region = input_data.split(" ")[0]
    else:
        dong = input_data
    
    if(dong not in list(clustered_dong["동 이름"])):
        yusa_dong_list.append("죄송합니다. 현재 해당하는 동에 대한 데이터는 수집 중입니다.")
        spring_response = search_sanggueon_gongtong(menu_data, [], input_data,yusa_dong_list,0,0)
        if(spring_response.get("status")=="success"):
            return jsonify({"status":"success",
                           "dong":"잘못된 입력",
                           "message":"전송은 성공했지만, 분석은 실패"})
        else:
            return jsonify({"status":"error",
                           "message":spring_response.get("error")})
        
    else:
        # 혹시 지역까지 입력해주면 더 확실하게 동을 찾아 줄 수 있음(동 이름이 겹치는 경우가 많음...)
        if(clustered_dong["지역"].str.contains(region).sum()>0):
            region_data = clustered_dong[clustered_dong["지역"].str.contains(region)]
            dong_data = region_data[region_data["동 이름"]==dong]
        else:
            dong_data = clustered_dong[clustered_dong["동 이름"]==dong]
    
        dong_pk = int(dong_data["법정동 번호(PK)"].values[0])
        yusa_dong_number = yusa_dong(clustered_dong, dong_pk)
        
        for i in range(len(yusa_dong_number)):
            yusa_dong_data=clustered_dong[clustered_dong["법정동 번호(PK)"]==yusa_dong_number[i]]
            yusa_dong_name=yusa_dong_data["동 이름"].values[0]
            print(yusa_dong_name)
            gongtong_menus, dong_gongtong_cnt, yusa_dong_gongtong_cnt = yusasang_gongtong(cafe_menu_data, dong, yusa_dong_name)
            spring_response_gongtong = search_sanggueon_gongtong(menu_data, gongtong_menus,dong, yusa_dong_name, dong_gongtong_cnt, yusa_dong_gongtong_cnt)
            
            if(spring_response_gongtong.get("status")!="success"):
                return jsonify({"status":"error",
                               "dong":yusa_dong_name,
                               "message":spring_response_gongtong.get("error")})
            
            only_menus, dong_only_cnt, yusa_dong_only_cnt = yusasang_only(cafe_menu_data, dong, yusa_dong_name)
            spring_response_only = search_sanggueon_only(menu_data, only_menus, dong, yusa_dong_name, dong_only_cnt, yusa_dong_only_cnt)
            
            if(spring_response_only.get("status")!="success"):
                return jsonify({"status":"error",
                               "dong":yusa_dong_name,
                               "message":spring_response_only.get("error")})
        
        return jsonify({"status":"success",
                       "dong":dong})
        
    

# 스프링 부트로부터 키워드 전달받아 분석
@app.route("/analyze-keyword", methods=["POST"])
def analyze_keyword():
    # 요청 본문 데이터 확인 : 키워드
    data = request.get_json()
    keyword = data.get("keyword", "")
    recommanded = input_simil(menu_data, keyword)  # 추천 메뉴 리스트

    # 스프링 서버로 요청 보내기
    spring_response = keyword_menu_send_to_spring(menu_data, keyword, recommanded)

    # 스프링 응답 확인
    if spring_response.get("status") == "success":
        return jsonify({
            "status": "success",
            "keyword": spring_response.get("keyword"),
            "recommanded": spring_response.get("recommanded"),
            "menuInfo": spring_response.get("menuInfo"),
            "menuIngred": spring_response.get("menuIngred")
        })
    else:
        return jsonify({
            "status": "error",
            "keyword": keyword,
            "message": spring_response.get("error")
        })

# 스프링 부트로부터 메뉴 전달받아 분석
@app.route("/analyze-menu", methods=["POST"])
def analyze_menu():
    # 요청 데이터 확인 : 메뉴
    data = request.get_json()
    menu_name = data.get("menu","")
    print(menu_name)
    similar_menus = ingred_sim(menu_data, menu_name)
    
    spring_response = similar_menu_recommanded(menu_data, similar_menus, menu_name)
#     print(spring_response["similarMenus"])
    if(spring_response.get("status")=="success"):
        return jsonify({"status":"success",
                        "menu":menu_name,
                       "similarMenus":similar_menus})
    else:
        return jsonify({"status":"error",
                       "message":spring_response.get("error")})

@app.route("/sanggueon-gongtong", methods=["POST"])
def sanggueon_gongtong():
    data = request.get_json()
    sang1 = data.get("sang1", "")
    sang2 = data.get("sang2", "")
    
    # 공통 메뉴와 카운트 계산
    gongtong_menus, sang1_cnt, sang2_cnt = yusasang_gongtong(cafe_menu_data, sang1, sang2)
    print(gongtong_menus)
    
    # 공통 상권 검색
    spring_response = search_sanggueon_gongtong(menu_data, gongtong_menus, sang1, sang2, sang1_cnt, sang2_cnt)
    print(f"Spring Response: {spring_response}")  # 디버깅용

    # spring_response가 dict이므로 json() 호출 불필요
    if spring_response.get("status") == "success":
        return jsonify({
        "status": "success",
        "gongtongMenus": spring_response.get("gontongMenus"),
        "menuIngred": spring_response.get("menuIngred"),
        "menuInfo": spring_response.get("menuInfo"),
        "sang1": spring_response.get("sang1"),
        "sang2": spring_response.get("sang2"),
        "sang1MenuCnt": spring_response.get("sang1MenuCnt"),
        "sang2MenuCnt": spring_response.get("sang2MenuCnt"),
        "message": spring_response.get("message", "No message provided")
        })
    else:
        return jsonify({
            "status": "error",
            "message": spring_response.get("error")
        })

@app.route("/sanggueon-only", methods=["POST"])
def sanggueon_only():
    data=request.get_json()
    sang1=data.get("sang1","")
    sang2=data.get("sang2","")
    only_menus, sang1_cnt, sang2_cnt = yusasang_only(cafe_menu_data, sang1, sang2)
    print(only_menus)
    spring_response = search_sanggueon_only(menu_data, only_menus, sang1, sang2, sang1_cnt, sang2_cnt)
    if(spring_response.get("status")=="success"):
        return jsonify({"status":"success","onlyMenu":only_menus})
    else:
        return jsonify({"status":"error","message":spring_response.get("error")})


if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5001)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5001
 * Running on http://192.168.219.227:5001
Press CTRL+C to quit
127.0.0.1 - - [03/Jan/2025 20:18:27] "POST /sanggueon-gongtong HTTP/1.1" 200 -
127.0.0.1 - - [03/Jan/2025 20:18:27] "POST /sanggueon-only HTTP/1.1" 200 -


[27, 26, 25, 25, 25]
[21, 15, 10, 12, 9]
['딸기 스무디', '케일 주스', '과일 생크림 케이크', '블루베리 요거', '블루베리 베이글']
{'status': 'success', 'gongtongMenus': ['딸기 스무디', '케일 주스', '과일 생크림 케이크', '블루베리 요거', '블루베리 베이글'], 'sang1': '동명동', 'sang2': '광안동', 'sang1MenuCnt': [27, 26, 25, 25, 25], 'sang2MenuCnt': [21, 15, 10, 12, 9], 'menuInfo': ['[달콤함, 상큼함, 과일향, 부드러움, 사랑스러움, 새콤달콤한 과일향, 부드러운 텍스처, 사랑스러운 맛, 상쾌함, 활기찬 분위기]', '[건강함, 신선함, 쌉쌀함, 깔끔함, 자연스러움, 풀향의 부드러움, 자연의 맛, 상쾌한 텍스처]', '[신선함, 화려함, 달콤함, 촉촉한 식감, 생크림의 풍미, 신선한 과일, 가벼운 단맛, 화사한 디저트, 고급스러움, 만족감]', '[건강함, 새콤달콤, 과일향, 부드러움, 상쾌함, 과일의 자연미, 건강한 즐거움]', '[새콤달콤, 고소함, 쫄깃한 식감, 신선한 블루베리 풍미, 건강한 선택, 자연주의, 만족감, 촉촉함, 화사함, 사랑스러움]'], 'menuIngred': ["[ '딸기', '우유', '얼음', '설탕']", "[ '케일', '사과', '레몬즙', '물', '설탕']", "[ '케이크 시트, 생크림, 설탕, 과일 (딸기, 블루베리, 키위 등), 바닐라 추출물, 젤라틴']", "[ '블루베리', '플레인 요거트', '우유', '꿀', '얼음 (옵션)', '민트 잎 (옵션)']", "[ '베이글', '블루베리', '크림치즈', '슈가 시럽', '레몬 제스트']"]}
['연유 라떼', '우유 빙수', '그릭 요거', '과일 주스', '토마토 주스']
{'status': 'success', 'onlyMenus': ['연유 라떼', '우유 빙수', '그릭 요거', '과

127.0.0.1 - - [03/Jan/2025 20:18:59] "POST /sanggueon-gongtong HTTP/1.1" 200 -
127.0.0.1 - - [03/Jan/2025 20:18:59] "POST /sanggueon-only HTTP/1.1" 200 -


[15, 14, 14, 14, 14]['자스민 티', '디카페인 카라멜 마끼야또', '에스프레소 라떼', '바닐라 라떼 콜드브루', '말차']
{'status': 'success', 'onlyMenus': ['자스민 티', '디카페인 카라멜 마끼야또', '에스프레소 라떼', '바닐라 라떼 콜드브루', '말차'], 'sang1': '광안동', 'sang2': '동명동', 'sang1MenuCnt': [2, 4, 8, 6, 7], 'sang2MenuCnt': [102, 46, 46, 39, 39], 'menuInfo': ['[우아함, 허브향, 편안함, 자연스러움, 깔끔함, 은은한 꽃향, 부드러운 허브감, 우아한 풍미, 고요한 여운, 자연스러운 감미]', '[달콤함, 크리미함, 건강함, 부드러움, 만족감, 부드러운 캐러멜, 크리미한 텍스처, 가벼운 풍미, 달콤한 여운, 편안함]', '[진한 커피맛, 부드러움, 고급스러움, 풍미, 클래식, 진한 커피 맛, 부드러운 우유, 균형감, 풍부한 여운]', '[부드러움, 달콤함, 크리미함, 시원함, 고급스러움, 부드러운 바닐라 향, 깔끔한 커피, 청량감, 자연스러운 단맛, 균형 잡힌 맛]', '[쌉쌀함, 고소함, 전통적, 자연스러움, 깊은 맛, 전통적 풍미, 자연스러운 깊이, 깔끔한 여운]'], 'menuIngred': ["[ '자스민 티 잎', '뜨거운 물', '설탕', '레몬']", "[ '디카페인 커피 원두', '우유', '카라멜 시럽', '얼음']", "[ '에스프레소', '우유', '설탕 (옵션)', '바닐라 시럽 (옵션)']", "[ '콜드브루 커피', '우유', '바닐라 시럽', '얼음', '휘핑크림 (옵션)']", "[ '말차 가루', '우유', '설탕', '물', '바닐라 시럽 (옵션)']"]}

[57, 25, 37, 55, 27]
['패션후르츠 에이드', '딸기 요거', '망고 스무디', '얼그레이 밀크티', '유자 에이드']
{'status': 'success', 'gongtongMenus': ['패션후르

...

[15, 14, 14, 14, 14]['자스민 티', '디카페인 카라멜 마끼야또', '에스프레소 라떼', '바닐라 라떼 콜드브루', '말차']
{'status': 'success', 'onlyMenus': ['자스민 티', '디카페인 카라멜 마끼야또', '에스프레소 라떼', '바닐라 라떼 콜드브루', '말차'], 'sang1': '광안동', 'sang2': '동명동', 'sang1MenuCnt': [2, 4, 8, 6, 7], 'sang2MenuCnt': [102, 46, 46, 39, 39], 'menuInfo': ['[우아함, 허브향, 편안함, 자연스러움, 깔끔함, 은은한 꽃향, 부드러운 허브감, 우아한 풍미, 고요한 여운, 자연스러운 감미]', '[달콤함, 크리미함, 건강함, 부드러움, 만족감, 부드러운 캐러멜, 크리미한 텍스처, 가벼운 풍미, 달콤한 여운, 편안함]', '[진한 커피맛, 부드러움, 고급스러움, 풍미, 클래식, 진한 커피 맛, 부드러운 우유, 균형감, 풍부한 여운]', '[부드러움, 달콤함, 크리미함, 시원함, 고급스러움, 부드러운 바닐라 향, 깔끔한 커피, 청량감, 자연스러운 단맛, 균형 잡힌 맛]', '[쌉쌀함, 고소함, 전통적, 자연스러움, 깊은 맛, 전통적 풍미, 자연스러운 깊이, 깔끔한 여운]'], 'menuIngred': ["[ '자스민 티 잎', '뜨거운 물', '설탕', '레몬']", "[ '디카페인 커피 원두', '우유', '카라멜 시럽', '얼음']", "[ '에스프레소', '우유', '설탕 (옵션)', '바닐라 시럽 (옵션)']", "[ '콜드브루 커피', '우유', '바닐라 시럽', '얼음', '휘핑크림 (옵션)']", "[ '말차 가루', '우유', '설탕', '물', '바닐라 시럽 (옵션)']"]}

[57, 25, 37, 55, 27]
['패션후르츠 에이드', '딸기 요거', '망고 스무디', '얼그레이 밀크티', '유자 에이드']
{'status': 'success', 'gongtongMenus': ['패션후르

127.0.0.1 - - [03/Jan/2025 20:22:22] "POST /sanggueon-gongtong HTTP/1.1" 200 -
127.0.0.1 - - [03/Jan/2025 20:22:22] "POST /sanggueon-only HTTP/1.1" 200 -


[15, 14, 14, 14, 14]['자스민 티', '디카페인 카라멜 마끼야또', '에스프레소 라떼', '바닐라 라떼 콜드브루', '말차']
{'status': 'success', 'onlyMenus': ['자스민 티', '디카페인 카라멜 마끼야또', '에스프레소 라떼', '바닐라 라떼 콜드브루', '말차'], 'sang1': '광안동', 'sang2': '동명동', 'sang1MenuCnt': [2, 4, 8, 6, 7], 'sang2MenuCnt': [102, 46, 46, 39, 39], 'menuInfo': ['[우아함, 허브향, 편안함, 자연스러움, 깔끔함, 은은한 꽃향, 부드러운 허브감, 우아한 풍미, 고요한 여운, 자연스러운 감미]', '[달콤함, 크리미함, 건강함, 부드러움, 만족감, 부드러운 캐러멜, 크리미한 텍스처, 가벼운 풍미, 달콤한 여운, 편안함]', '[진한 커피맛, 부드러움, 고급스러움, 풍미, 클래식, 진한 커피 맛, 부드러운 우유, 균형감, 풍부한 여운]', '[부드러움, 달콤함, 크리미함, 시원함, 고급스러움, 부드러운 바닐라 향, 깔끔한 커피, 청량감, 자연스러운 단맛, 균형 잡힌 맛]', '[쌉쌀함, 고소함, 전통적, 자연스러움, 깊은 맛, 전통적 풍미, 자연스러운 깊이, 깔끔한 여운]'], 'menuIngred': ["[ '자스민 티 잎', '뜨거운 물', '설탕', '레몬']", "[ '디카페인 커피 원두', '우유', '카라멜 시럽', '얼음']", "[ '에스프레소', '우유', '설탕 (옵션)', '바닐라 시럽 (옵션)']", "[ '콜드브루 커피', '우유', '바닐라 시럽', '얼음', '휘핑크림 (옵션)']", "[ '말차 가루', '우유', '설탕', '물', '바닐라 시럽 (옵션)']"]}

[57, 25, 37, 55, 27]
['패션후르츠 에이드', '딸기 요거', '망고 스무디', '얼그레이 밀크티', '유자 에이드']
{'status': 'success', 'gongtongMenus': ['패션후르

127.0.0.1 - - [03/Jan/2025 20:23:15] "POST /sanggueon-only HTTP/1.1" 200 -
127.0.0.1 - - [03/Jan/2025 20:23:15] "POST /sanggueon-gongtong HTTP/1.1" 200 -


[15, 14, 14, 14, 14]
[57, 25, 37, 55, 27]
['패션후르츠 에이드', '딸기 요거', '망고 스무디', '얼그레이 밀크티', '유자 에이드']
{'status': 'success', 'gongtongMenus': ['패션후르츠 에이드', '딸기 요거', '망고 스무디', '얼그레이 밀크티', '유자 에이드'], 'sang1': '광안동', 'sang2': '동명동', 'sang1MenuCnt': [15, 14, 14, 14, 14], 'sang2MenuCnt': [57, 25, 37, 55, 27], 'menuInfo': ['[이국적, 새콤달콤, 청량감, 활기참, 과일향, 이국적 과일향, 상쾌함, 청량한 탄산]', '[달콤함, 상큼함, 과일향, 건강함, 부드러움, 사랑스러운 과일향, 새콤달콤, 부드러운 질감, 건강한 단맛, 활기참]', '[이국적, 달콤함, 과일향, 시원함, 상쾌함, 열대과일의 상큼함, 부드러움, 자연스러운 달콤함, 활기참, 신선한 질감]', '[우아함, 부드러움, 깊은 맛, 클래식, 고급스러움, 베르가못 향, 부드러운 크림, 우아한 맛, 조화로움, 깔끔한 끝맛]', '[상큼함, 청량감, 과일향, 산뜻함, 리프레시, 상큼한 감귤 향, 청량한 탄산, 자연스러운 단맛, 활기참, 산뜻한 마무리]'], 'menuIngred': ["[ '패션후르츠', '탄산수', '설탕', '레몬', '얼음']", "[ '딸기', '플레인 요거트', '우유', '꿀', '얼음 (옵션)', '민트 잎 (옵션)']", "[ '망고', '우유', '바닐라 아이스크림', '설탕', '얼음']", "[ '얼그레이 티백, 우유, 설탕, 얼음']", "[ '유자청', '탄산수', '얼음', '민트 잎 (옵션)']"]}
['자스민 티', '디카페인 카라멜 마끼야또', '에스프레소 라떼', '바닐라 라떼 콜드브루', '말차']
{'status': 'success', 'onlyMenus': ['자스민 티', '디카페인 카라멜 마끼야또', '에스프레소 라떼'

In [67]:
# 유사 상권 탐색
def yusa_dong(clustered_dong, dong_pk):
    target_dong_index = clustered_dong[clustered_dong["법정동 번호(PK)"]==dong_pk].index[0]

    same_cluster_dong = clustered_dong[clustered_dong["군집"]==clustered_dong.iloc[target_dong_index,-1]]
    same_cluster_dong.reset_index(drop=True, inplace=True)
    target_dong_index = same_cluster_dong[same_cluster_dong["법정동 번호(PK)"]==dong_pk].index[0]
    
    simil_data = same_cluster_dong.iloc[:,3:-1]
    simil_data_scaled = normalizer.fit_transform(simil_data)

    train_tensor = torch.tensor(simil_data_scaled, dtype=torch.float64)
    euclidean_distances = torch.cdist(train_tensor[target_dong_index].unsqueeze(0), train_tensor).squeeze()
    top_idx = np.argsort(euclidean_distances)[1:10]

    return [int(same_cluster_dong["법정동 번호(PK)"].iloc[int(idx)]) for idx in top_idx]

In [69]:
def data_tokenizing(data):
    data["ingred"] = data["ingred"].replace('\'', '', regex=True)
    kiwi = Kiwi()
    for i in range(len(data["info"])):
        data.iloc[i, 1] = kiwi_tokenizer(data["info"][i], tokenizer=kiwi)
    return data

In [11]:
def ingred_sim(data, menu_name, num=10):
    # info_simil 함수로 유사한 메뉴 추출
    info_sim = info_simil(data, menu_name, 50)
    
    # 유사한 메뉴가 없는 경우 처리
    if info_sim.empty or len(info_sim["menu"]) == 0:
        return ["유사한 메뉴가 존재하지 않습니다..."]

    # TF-IDF 변환
    tfidfv = TfidfVectorizer().fit_transform(info_sim["tokenized_info"])

    # 코사인 유사도 계산
    ingred_simil = cosine_similarity(tfidfv[0], tfidfv)[0]

    # 자기 자신 제외하고 유사도 순으로 정렬
    ingred_simil_index = np.argsort(ingred_simil)[::-1][1:num + 1]

    # 유사도가 낮은 경우 처리
    if len(ingred_simil_index) == 0 or ingred_simil[ingred_simil_index[0]] < 0.2:
        return ["유사한 메뉴가 존재하지 않습니다..."]

    # 결과 반환
    return list(info_sim.iloc[ingred_simil_index]["menu"])

In [13]:
def info_simil(data, menu_name, num=10):
    # 메뉴 이름이 유효한지 확인
    if menu_name not in data["menu"].values:
        print("메뉴 이름이 잘못되었습니다.")
        return pd.DataFrame()  # 빈 데이터프레임 반환

    # 선택된 메뉴의 인덱스 가져오기
    pick_idx = data[data["menu"] == menu_name].index[0]

    # TF-IDF 변환
    tfidfv = TfidfVectorizer().fit_transform(data["tokenized_info"])

    # 코사인 유사도 계산
    cosine_simil = cosine_similarity(tfidfv[pick_idx], tfidfv)[0]

    # 유사도를 기준으로 정렬 (내림차순)
    simil_index = np.argsort(cosine_simil)[::-1]

    # 자기 자신 제외하고 상위 num개 선택
    simil_index = simil_index[1:num + 1]

    # 유사도가 일정 수준 이하인 경우 처리
    if len(simil_index) == 0 or cosine_simil[simil_index[0]] < 0.2:
        print("유사한 메뉴가 존재하지 않습니다.")
        return pd.DataFrame()  # 빈 데이터프레임 반환

    # 결과 반환
    return data.iloc[simil_index]

In [15]:
def input_simil(data, input_data, num=10):
    input_token = kiwi_tokenizer(input_data)
    text_vector = np.array(data["menu"] + " " + data["tokenized_info"])
    t_vector = np.insert(text_vector, 0, input_token)

    tfidfv = TfidfVectorizer().fit_transform(t_vector)
    cosine_simil = cosine_similarity(tfidfv[0], tfidfv)[0]

    simil_index = np.flip(np.argsort(cosine_simil[1:]))  # Exclude self-similarity

    # Lower threshold to capture more matches
    threshold = 0.15
    if cosine_simil[simil_index[0] + 1] < threshold:
        return ["유사한 메뉴가 존재하지 않습니다..."]
    else:
        top_matches = data.iloc[simil_index[:num], 0]
        return list(top_matches)

In [17]:
def kiwi_tokenizer(sentence, tokenizer=Kiwi(typos="basic")):
    sentence = sentence.replace(" ", "")
    stopwords = Stopwords()
    get_tags = ["XR", "NNG", "VA"]
    sentence = tokenizer.space(sentence)

    kiwi_result = tokenizer.tokenize(
        sentence, normalize_coda=True, stopwords=stopwords)
    token_list = []
    for token in range(len(kiwi_result)):
        if (kiwi_result[token].tag in get_tags):
            token_list.append(kiwi_result[token].form)
    token_str = " ".join(token_list)
    return token_str

In [57]:
def keyword_menu_send_to_spring(menu_data, keyword, recommanded):
    spring_url = "http://localhost:8087/api/receive-keyword-menu"
    
    menu_info = [f'[{menu_data[menu_data["menu"]==menu]["info"].values[0]}]' for menu in recommanded]
    menu_ingred = [f'[{menu_data[menu_data["menu"]==menu]["ingred"].values[0]}]' for menu in recommanded]
    
    # 전송할 payload 준비
    payload = {
        "status": "success",
        "keyword": keyword,
        "recommanded": recommanded,
        "menuInfo": menu_info,
        "menuIngred": menu_ingred
    }
    
    headers = {"Content-Type": "application/json"}
    print("Payload to be sent:", payload)  # payload 출력
    try:
        response = requests.post(spring_url, json=payload, headers=headers)
        response.raise_for_status()  # 예외가 발생하면 여기서 처리
        print(response.status_code)  # 응답 코드 출력
        print(response.json())  # 응답 내용 출력
        return response.json()  # 응답 JSON을 반환
    except Exception as e:
        # 예외 처리
        print(f"스프링 부트 전송 실패: {str(e)}")
        return {"status": "error", "error": str(e)}  # 실패한 경우 error 메시지 반환

In [23]:
# 분석한 유사 상권 정보 전달
def similar_dong_send_to_spring(similar_dongs):
    spring_url = "http://localhost:8087/api/receive-similar-dongs"
    # 스프링에 dong이라는 이름으로 dong_name 데이터 전송
    payload = {"similarDongs": similar_dongs,
               "status": "success"
              }
    # json 형식으로 전송
    headers = {"Content-Type": "application/json"}
    
    try:
        response = requests.post(spring_url, json=payload, headers=headers)
        response.raise_for_status()  # HTTP 오류 있는지 확인 => 예외 발생
        return {"status": "success", "message": "스프링으로 전송 완료"}
    except Exception as e:
        # 예외처리
        print(f"스프링 전송 실패: {str(e)}")
        return {"status": "error", "error": str(e)}

In [21]:
# 유사 메뉴 분석 결과 스프링으로 전송
def similar_menu_recommanded(menu_data, similar_menus, menu_name):
    spring_url="http://localhost:8087/api/receive-similar-menus"
    
    menu_info = [f'[{menu_data[menu_data["menu"]==menu]["info"].values[0]}]' for menu in similar_menus]
    menu_ingred = [f'[{menu_data[menu_data["menu"]==menu]["ingred"].values[0]}]' for menu in similar_menus]
    
    payload={"status":"success",
            "similarMenus":similar_menus,
            "menu": menu_name,
             "menuInfo": menu_info,
             "menuIngred":menu_ingred
            }
        
    headers={"Content-Type" : "application/json"}
    print(payload)
    
    try:
        response = requests.post(spring_url, json=payload, headers=headers)
        response.raise_for_status()
        return {"status":"success","message":"스프링으로 전송 완료"}
    except Exception as e:
        print(f"스프링 전송 실패: {str(e)}")
        return {"status" : "error", "error":str(e)}

In [76]:
# 상권 공통 메뉴 분석 결과 스프링으로 전송
def search_sanggueon_gongtong(menu_data, gongtong_menus, sang1, sang2, sang1_ratio, sang2_ratio):
    spring_url="http://localhost:8087/api/receive-sanggueon-gongtong"
    
    menu_info = [f'[{menu_data[menu_data["menu"]==menu]["info"].values[0]}]' for menu in gongtong_menus if menu in list(menu_data["menu"])]
    menu_ingred = [f'[{menu_data[menu_data["menu"]==menu]["ingred"].values[0]}]' for menu in gongtong_menus if menu in list(menu_data["menu"])]
    
    payload={"status":"success",
            "gongtongMenus" : gongtong_menus,
             "sang1" : sang1,
             "sang2":sang2,
             "sang1MenuCnt":sang1_ratio,
             "sang2MenuCnt":sang2_ratio,
             "menuInfo":menu_info,
             "menuIngred":menu_ingred
            }
        
    headers={"Content-Type" : "application/json"}
    print(payload)
    
    try:
        response = requests.post(spring_url, json=payload, headers=headers)
        response.raise_for_status()
        return response.json()
    except Exception as e:
        print(f"스프링 전송 실패: {str(e)}")
        return {"status" : "error", "error":str(e)}

In [27]:
# 상권2만 언급이 많은 메뉴 전송
def search_sanggueon_only(menu_data, only_menus, sang1, sang2, sang1_ratio, sang2_ratio):
    spring_url="http://localhost:8087/api/recieve-sanggueon-only"
    
    menu_info = [f'[{menu_data[menu_data["menu"]==menu]["info"].values[0]}]' for menu in only_menus if menu in list(menu_data["menu"])]
    menu_ingred = [f'[{menu_data[menu_data["menu"]==menu]["ingred"].values[0]}]' for menu in only_menus if menu in list(menu_data["menu"])]
    
    payload={"status":"success",
             "onlyMenus":only_menus,
             "sang1":sang1,
             "sang2":sang2,
             "sang1MenuCnt":sang1_ratio,
             "sang2MenuCnt":sang2_ratio,
             "menuInfo":menu_info,
             "menuIngred":menu_ingred
            }
    headers={"Content-Type":"application/json"}
    print(payload)
    
    try:
        response=requests.post(spring_url,json=payload,headers=headers)
        response.raise_for_status()
        return{"status":"success","message":"스프링으로 전송 완료"}
    except Exception as e:
        print(f"스프링 전송 실패: {str(e)}")
        return {"status":"error", "error":str(e)}

In [29]:
# 좌표를 통해 법정동 번호 조회
def get_dong_number(latitude, longitude, dong_data=clustered_dong):
    url = f"https://dapi.kakao.com/v2/local/geo/coord2address.json?x={longitude}&y={latitude}"
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}

    response = requests.get(url, headers=headers)

    # 200번 : 통신 성공
    if response.status_code == 200:
        address_info = response.json()
        # 카카오 api에 해당 좌표 정보가 있는지 확인
        if address_info['documents']:
            addr = address_info['documents'][0]['address']['address_name']
            region = addr.split(" ")[0]
            gu = addr.split(" ")[1]
            dong = addr.split(" ")[2]

            region_dong = dong_data[dong_data["지역"].str.contains(region)]
            dong_number = list(
                region_dong[region_dong["동 이름"] == dong].head(1)["법정동 번호(PK)"])[0]
            return dong_number
    return None

In [31]:
# 두 상권 중 다른 상권에만 유독 더 많이 파는 메뉴
def yusasang_only(data, sang1, sang2):
    sang1data = data[data["dong"]==sang1]
    sang2data = data[data["dong"]==sang2]
    
    sang1_count = Counter(sang1data['menu_keyword'])
    sang2_count = Counter(sang2data['menu_keyword'])

    sang1_menu = list(sang1_count.keys())
    sang1_cnt = sang1_count.values()
    sang1_cnt_list = list(sang1_cnt)
    sang1_rat = []
    sang1_sum = 0

    sang2_menu = list(sang2_count.keys())
    sang2_cnt = sang2_count.values()
    sang2_cnt_list = list(sang2_cnt)
    sang2_rat = []
    sang2_sum = 0

    for i in range(len(sang1_cnt)):
        sang1_sum+=sang1_cnt_list[i]
    for i in range(len(sang2_cnt)):
        sang2_sum+=sang2_cnt_list[i]

    sang1_rat = np.array([r/sang1_sum for r in sang1_cnt_list])
    sang2_rat = np.array([r/sang2_sum for r in sang2_cnt_list])

    sang1_most_rat = np.flip(np.argsort(sang1_rat)[:])
    sang2_most_rat = np.flip(np.argsort(sang2_rat)[:])

    sang1_most = [sang1_menu[menu_i] for menu_i in sang1_most_rat]
    sang2_most = [sang2_menu[menu_i] for menu_i in sang2_most_rat]

    only_sang2 = []
    sang1_ratio = []
    sang2_ratio = []
    popularity = len(sang1_most)//5
    
    for i in range(len(sang2_most)):
        if(sang2_most[i] not in sang1_most[:popularity]):
            only_sang2.append(sang2_most[i])
            try:
                sang1_ratio.append(sang1_cnt_list[sang1_menu.index(sang2_most[i])])
            except:
                sang1_ratio.append(0)
            sang2_ratio.append(sang2_cnt_list[sang2_most_rat[i]])
        if(len(only_sang2) == 5):
            break
    if(len(only_sang2) < 3):
        only_sang2.append("두 상권의 거의 모든 메뉴가 유사합니다.")
        sang1_ratio.append(0)
        sang2_ratio.append(0)

    return [only_sang2, sang1_ratio, sang2_ratio]

In [33]:
#두 상권 모두 많이 파는 메뉴
def yusasang_gongtong(data, sang1, sang2):
    sang1data = data[data["dong"]==sang1]
    sang2data = data[data["dong"]==sang2]
    
    sang1_count = Counter(sang1data['menu_keyword'])
    sang2_count = Counter(sang2data['menu_keyword'])

    sang1_menu = list(sang1_count.keys())
    sang1_cnt = sang1_count.values()
    sang1_cnt_list = list(sang1_cnt)
    sang1_rat = []
    sang1_sum = 0

    sang2_menu = list(sang2_count.keys())
    sang2_cnt = sang2_count.values()
    sang2_cnt_list = list(sang2_cnt)
    sang2_rat = []
    sang2_sum = 0

    for i in range(len(sang1_cnt)):
        sang1_sum+=sang1_cnt_list[i]
    for i in range(len(sang2_cnt)):
        sang2_sum+=sang2_cnt_list[i]

    sang1_rat = np.array([r/sang1_sum for r in sang1_cnt_list])
    sang2_rat = np.array([r/sang2_sum for r in sang2_cnt_list])

    sang1_most_rat = np.flip(np.argsort(sang1_rat)[:])
    sang2_most_rat = np.flip(np.argsort(sang2_rat)[:])

    sang1_most = [sang1_menu[menu_i] for menu_i in sang1_most_rat]
    sang2_most = [sang2_menu[menu_i] for menu_i in sang2_most_rat]
    
    sang1_ratio = []
    sang2_ratio = []
    popularity = len(sang2_most) // 10
    
    gongtong_sang = []
    for i in range(popularity, len(sang1_most)):
        if(sang1_most[i] in sang2_most[:popularity*2]):
            gongtong_sang.append(sang1_most[i])
            sang1_ratio.append(sang1_cnt_list[sang1_most_rat[i]])
            sang2_ratio.append(sang2_cnt_list[sang2_menu.index(sang1_most[i])])
        if(len(gongtong_sang)==5):
            break
    if(len(gongtong_sang) < 3):
        gongtong_sang.append("두 상권의 메뉴는 거의 유사하지 않습니다.")
    print(sang1_ratio)
    print(sang2_ratio)
    return [gongtong_sang, sang1_ratio, sang2_ratio]

In [35]:
# 현재 상권에서 주로 파는 메뉴
def sang_menu(data, sang, no_menus=[]):
    sangdata = data[data["dong"]==sang]
    
    sang_count = Counter(sangdata['menu_keyword'])
    sang_most = sang_count.most_common(len(sang_count))
    
    sang_best_menu = []
    for i in range(5, len(sang_most)):
        if(sang_most[i][0] in no_menus):
            continue
        sang_best_menu.append(sang_most[i][0])
        if(len(sang_best_menu)==5):
            break
    if(len(sang_best_menu)<3):
        sang_best_menu.append("해당 상권은 분석할 수 없습니다. 사유 : 상권 내 카페 수 부족")
    return sang_best_menu

In [187]:
recommanded2 = ingred_sim(menu_data, "닭가슴살 샌드위치")
recommanded2

['햄 치즈 치아바타',
 '치킨 샌드위치',
 '샐러드 샌드위치',
 '반미 샌드위치',
 '미트볼 샌드위치',
 '치즈 치아바타',
 '햄치즈 바게트',
 '함박 샌드위치',
 '갈릭 베이글',
 '치즈 샌드위치']

In [194]:
sang_menu(cafe_menu_data, "동명동")

['쇼콜라 라떼', '딸기 라떼', '레몬 에이드', '얼그레이 티', '돌체 라떼']

In [99]:
sys.version

'3.11.9 | packaged by Anaconda, Inc. | (main, Apr 19 2024, 16:40:41) [MSC v.1916 64 bit (AMD64)]'

In [100]:
from kiwipiepy import Kiwi

# Kiwi 객체 생성
kiwi = Kiwi()

# 텍스트 입력
input_text = "안녕하세요! 파이썬을 배우고 있습니다."

# 토큰화
tokenized = kiwi.no(input_text)

# 결과 출력
for word in tokenized:
    print(word.form)

안녕
하
세요
!
파이썬
을
배우
고
있
습니다
.
